# P3 Examen
- **Name**: Elias De Hondt
- **Class**: ISB204B
- **Course**: Data Science 2
- **Date**: 09/03/2024
- **StudentID**: 0160712-80

> You hand in a **completed notebook**, so the **results of the calculations are also included**. Results should **not** be rounded.

In [36]:
# Import statements:s
from termcolor import colored # type: ignore                                          # Colored text
from random import Random  # type: ignore                                             # Random number generator
import math  # type: ignore                                                           # Mathematical functions
import pandas as pd  # type: ignore                                                   # Data manipulation
import numpy as np  # type: ignore                                                    # Scientific computing
import matplotlib.pyplot as plt  # type: ignore                                       # Data visualization
from scipy.stats import binom as binomial  # type: ignore                             # Binomial distribution
from scipy.stats import norm as normal  # type: ignore                                # Normal distribution
from scipy.stats import poisson as poisson  # type: ignore                            # Poisson distribution
from scipy.stats import t as student  # type: ignore                                  # Student distribution
from scipy.stats import chi2  # type: ignore                                          # Chi-squared distribution
from scipy.stats import ttest_1samp  # type: ignore                                   # One-sample t-test
from scipy.stats import chisquare  # type: ignore                                     # Chi-squared test
from scipy.special import comb  # type: ignore                                        # Combinations
from mlxtend.frequent_patterns import apriori  # type: ignore                         # Apriori algorithm
from mlxtend.frequent_patterns import fpgrowth  # type: ignore                        # FP-growth algorithm
from mlxtend.frequent_patterns import association_rules  # type: ignore               # Association rules
from mlxtend.preprocessing import TransactionEncoder  # type: ignore                  # Transaction encoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  # type: ignore  # Discriminant Analysis
from tensorflow import keras  # type: ignore                                          # Deep Learning library
from tensorflow.keras import Model  # type: ignore                                    # Model class
from tensorflow.keras.layers import Input, Dense, BatchNormalization  # type: ignore  # Layers
from tensorflow.keras.utils import to_categorical  # type: ignore                     # One-hot encoding
from tensorflow.keras.optimizers import Adam  # type: ignore                          # Optimizer
from livelossplot import PlotLossesKeras  # type: ignore                              # Live plot
from keras.src.optimizers import RMSprop  # type: ignore                              # Optimizer
from sklearn.model_selection import train_test_split  # type: ignore                  # Train-test split
from sklearn.metrics import roc_auc_score # type: ignore                              # ROC AUC score
from simanneal import Annealer  # type: ignore                                        # Simulated Annealing
from inspyred import ec  # type: ignore                                               # Evolutionary Computation
import warnings  # type: ignore                                                       # Disable warnings
warnings.filterwarnings("ignore")                                                     # Disable warnings
outputColor = "blue"                                                                  # Color for the output

In [37]:
# Function definitions:
def rule_filter(row, min_len, max_len):
    length = len(row['antecedents']) + len(row['consequents'])
    return min_len <= length <= max_len

def get_item_list (string):
    items = string [1:-1]
    return items.split(';')

def no_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    I = Q3 - Q1
    low = Q1 - 1.5 * I
    high = Q3 + 1.5 * I
    outliers = data[(data < low) | (data > high)]
    
    print("Low: ",low)
    print("High:",high)
    print("Len: ",len(data))
    print("Outliers:", outliers.values, "\n")
    return data[(data >= low) & (data <= high)]

def plot_confidence_interval(population_size, sample_mean, sample_standard_deviation, degrees_freedom, plot_factor):
    margin_of_error = plot_factor * sample_standard_deviation / np.sqrt(population_size)
    lower_bound = sample_mean - margin_of_error
    upper_bound = sample_mean + margin_of_error

    plt.figure(figsize=(10, 6))
    x_axis = np.linspace(sample_mean - 3 * sample_standard_deviation, sample_mean + 3 * sample_standard_deviation, 1000)
    y_axis = student.pdf(x_axis, degrees_freedom, loc=sample_mean, scale=sample_standard_deviation / np.sqrt(population_size))

    plt.plot(x_axis, y_axis, label='t-distribution')
    plt.axvline(lower_bound, color='red', linestyle='--', label='Lower Bound')
    plt.axvline(upper_bound, color='blue', linestyle='--', label='Upper Bound')
    plt.axvline(sample_mean, color='green', linestyle='-', label='Sample Mean')

    plt.fill_betweenx(y_axis, lower_bound, upper_bound, where=(x_axis >= lower_bound) & (x_axis <= upper_bound), color='orange', label='Confidence Interval')

    plt.title('Confidence Interval Plot')
    plt.xlabel('Sample Mean')
    plt.ylabel('Probability Density Function')
    plt.legend()
    plt.grid(True)
    plt.show()

In [38]:
bevolkingData = pd.read_csv('../Data/Bevolking.csv', delimiter=';', decimal='.', index_col='id')
display(bevolkingData)

,age,sex,region,income,married,children,car,fiber,iphone,linux
id,,,,,,,,,,
ID12101,48,FEMALE,INNER_CITY,17546.00,False,1,False,True,False,True
ID12102,40,MALE,TOWN,30085.10,True,3,True,True,True,False
ID12103,51,FEMALE,INNER_CITY,16575.40,True,0,True,True,False,False
ID12104,23,FEMALE,TOWN,20375.40,True,3,False,True,False,False
ID12105,57,FEMALE,RURAL,50576.30,True,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
ID12696,61,FEMALE,INNER_CITY,47025.00,False,2,True,True,True,False
ID12697,30,FEMALE,INNER_CITY,9672.25,True,0,True,True,False,False
ID12698,31,FEMALE,TOWN,15976.30,True,0,True,False,False,True


## Questions 1

In [39]:
# Probability Mass Function

probability=len(bevolkingData[ (bevolkingData["region"] == "TOWN") | (bevolkingData["car"] == True)]) / len(bevolkingData)
print(colored(f"The probability is as follows: {round(probability, 4)} or {round(probability * 100, 4)}%.", outputColor))

The probability is as follows: 0.63 or 63.0%.


## Questions 2

In [40]:
# Technique used: Laplace

probability = (1 / 4 * 0.001) + (1 / 4 * 0.002) + (2 / 4 * 0.003)
print(colored(f"The probability is as follows: {round(probability, 4)} or {round(probability * 100, 4)}%.", outputColor))

The probability is as follows: 0.0023 or 0.225%.


## Questions 3

In [41]:
# Technique used: The Binomial Distribution

true = len(bevolkingData[(bevolkingData["married"] == True)])
total = len(bevolkingData)

k = 7                 # Probability of exactly k successes
n = 10                # Number of trials
p = true / total      # Probability of success in each trial
probability = 1 - binomial.cdf(k, n, p)
print(colored(f"The probability is as follows: {round(probability, 4)} or {round(probability * 100, 4)}%.", outputColor))

The probability is as follows: 0.2838 or 28.377%.


## Questions 4

In [42]:
# Technique used: The Poisson Distribution

x = 3      # Number of events
y = 10     # Average number of events
probability = 1 - poisson.cdf(x, y)
print(colored(f"The probability is as follows: {round(probability, 4)} or {round(probability * 100, 4)}%.", outputColor))

The probability is as follows: 0.9897 or 98.9664%.


## Questions 5

In [43]:
# Technique used: Normal Distribution

loc=bevolkingData['income'].mean()
scale=bevolkingData['income'].std()
x = 65000
probability = 1 - normal.cdf(x, loc, scale)
print(colored(f"The probability is as follows: {round(probability, 4)} or {round(probability * 100, 4)}%.", outputColor))

The probability is as follows: 0.0018 or 0.1835%.


## Questions 6

In [48]:
a = 0.05
x_bar = bevolkingData['age'].mean()
s = bevolkingData['age'].std()
n = len(bevolkingData['age'])
df = n - 1
p = 1 - a

interval=student.interval(confidence=p, df=df, loc=x_bar, scale=s/math.sqrt(n))
print(colored(f"Confidence Interval: {interval}", outputColor))

Confidence Interval: (41.23844813332852, 43.55155186667149)


## Questions 7

In [51]:
income = no_outliers(bevolkingData['income'])
mu = 6500
a = 0.05
data = ttest_1samp(income, mu)
p_value = data.pvalue
print(colored(f"P Value: {p_value}", outputColor))
print(colored(f"5% {income.mean() - (income.mean() / 100 * 5)}", outputColor))

if p_value < a:
    print(colored("Nee", outputColor))
else:
    print(colored("Ja", outputColor))

print(colored(f"Het gemiddelde is niet {mu} wat {a} is groter dan de {p_value}." , outputColor))

Low:  -11097.762499999993
High: 64534.937499999985
Len:  600
Outliers: [] 

P Value: 6.179025530039627e-171
5% 26147.82965583333
Nee
Het gemiddelde is niet 6500 wat 0.05 is groter dan de 6.179025530039627e-171.


## Questions 8

In [54]:
# Assuming bevolkingData is a DataFrame and 'children' is a column in it
children = bevolkingData['children']

# Initialize counts
children1 = 0
children2 = 0
children3 = 0

# Count occurrences
for i in range(len(children)):
    if children.iloc[i] == 1:
        children1 += 1
    elif children.iloc[i] == 2:
        children2 += 1
    elif children.iloc[i] == 3:
        children3 += 1

# Calculate total
total = children1 + children2 + children3

print(colored(f"Amount of children1: {children1}", outputColor))
print(colored(f"Amount of children2: {children2}", outputColor))
print(colored(f"Amount of children3: {children3}", outputColor))
print(colored(f"Total: {total}", outputColor))

# Define measured and expected values
measured_values = [children1, children2, children3]
expected_values = [total * 0.45, total * 0.20, total * 0.10]

# Adjust expected values to ensure their sum matches the total
scaling_factor = total / sum(expected_values)
expected_values = [ev * scaling_factor for ev in expected_values]

# Perform chi-square test
data = chisquare(measured_values, expected_values)

print(colored(f"P Value: {data.pvalue}", outputColor))

# Significance level
a = 0.05

if data.pvalue < a:
    print(colored("Nee", outputColor))
else:
    print(colored("Ja", outputColor))

Amount of children1: 135
Amount of children2: 134
Amount of children3: 68
Total: 337
P Value: 7.45789993632972e-13
Nee


## Questions 9

## Questions 10

In [59]:
# Apriori algorithm
support = 0.2
min_threshold = 0.7
bevolkingData_dummies=bevolkingData.drop(columns=['age', 'sex', 'region', 'income', 'married', 'children'])

item_sets_apriori=apriori(bevolkingData_dummies, min_support=support, use_colnames=True)

rules_apriori=association_rules(item_sets_apriori, metric='confidence', min_threshold=min_threshold)
rules_apriori=rules_apriori.drop(columns=['leverage', 'conviction', 'zhangs_metric'])

print(colored(f"Wat (linux) heeft de grootsten confidence bij fiber.", outputColor))
display(rules_apriori.sort_values(by='confidence', ascending=False).head(1)['antecedents'])

Wat (linux) heeft de grootsten confidence bij fiber.


2    (linux)
Name: antecedents, dtype: object